In [1]:
# The database object is responsible for maintaining the
# snapshots (versions) of datasets that are manipulated using
# openclean operators in this notebook.

from openclean_notebook import DB

# Define a base directory on the local file system where all
# data is stored. The create=True flag will erase any data
# that previously exists in the base directory.
db = DB(basedir='.openclean', create=True)

In [2]:
from openclean.data.load import dataset

#
# Load full dataset 'VDH-COVID-19-PublicUseDataset-Cases' from the data.virginia.gov portal.

df = db.load_dataset(source='./data/bre9-aqqr.tsv.gz', name='covid-cases').checkout()

In [3]:
# Add a user-defined function that operates on two columns and
# that takes an additional parameter as input.
#
# When registering the function we can specify the number of
# input columns the function operates on. The convention is that
# the first n arguments of the registered function will receive
# their values from the n input columns that the user selects.
# Additional parameters will be called as keyword arguments.
#
# If the collables argument is not given when the function is
# registered the names of the first n function arguments are
# used as the defaults.

from openclean.engine.object.function import String

@db.register.eval(
    name='concat',
    label='Concat Columns',
    description='Concatenate values from two columns',
    columns=2,
    collabels=['Left Column', 'Right Column'],
    parameters=[String(name='delim', label='Delimiter', default=':')]
)
def concat_columns(value1, value2, delim):
    """Concatenate two values with the given delimiter."""
    return '{}{}{}'.format(value1, delim, value2)

In [4]:
# This is an example for a user-defined function that operates
# on a single input column but generates a pair of values for
# two output columns.
#
# When registering the function we can specify the number of
# output columns using the 'outputs' parameter.

from openclean.engine.object.function import String

@db.register.eval(
    name='split',
    label='Split Column',
    description='Split values from one column',
    columns=1,
    collabels=['Input Column'],
    outputs=2,
    parameters=[String(name='delim', label='Split delimiter', default=':')]
)
def split_column(value, delim):
    """Split column value on the first occurrence of the given delimiter."""
    if delim in value:
        pos = value.find(delim)
        return value[:pos], value[pos + len(delim):]
    else:
        return value, None

In [5]:
# Print serialization of function library that will be available to the Spreadsheet view.

import json

print(json.dumps(db.library_dict(), indent=4))

{
    "functions": [
        {
            "name": "lower",
            "namespace": "string",
            "columns": 1,
            "columnLabels": null,
            "outputs": 1,
            "parameters": []
        },
        {
            "name": "upper",
            "namespace": "string",
            "columns": 1,
            "columnLabels": null,
            "outputs": 1,
            "parameters": []
        },
        {
            "name": "capitalize",
            "namespace": "string",
            "columns": 1,
            "columnLabels": null,
            "outputs": 1,
            "parameters": []
        },
        {
            "name": "concat",
            "namespace": null,
            "label": "Concat Columns",
            "description": "Concatenate values from two columns",
            "columns": 2,
            "columnLabels": [
                "Left Column",
                "Right Column"
            ],
            "outputs": 1,
            "parameters": [
           

In [6]:
df.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths
0,03/17/2020,51001,Accomack,Eastern Shore,0,0,0
1,03/17/2020,51003,Albemarle,Thomas Jefferson,0,0,0
2,03/17/2020,51005,Alleghany,Alleghany,0,0,0
3,03/17/2020,51007,Amelia,Piedmont,0,0,0
4,03/17/2020,51009,Amherst,Central Virginia,0,0,0


In [7]:
# Apply the concat function to update column 'Locality' with the
# concatenated values from column 'Locality' and 'FIPS'.
#
# Note that the 'columns' argument of the update function refers
# to the column(s) that are being updated. If the updated column
# is different from the input columns (like in this example) the
# input column names have to be specified using the 'sources' parameter.
# Additional arguments that are passed evaluated update function
# (i.e., in this case 'delim') are specified as a dictionary via
# the 'args' parameter. 

df = db\
    .dataset('covid-cases')\
    .update(
        columns='Locality',  # Column that is being updated
        func=db.library.functions().get(name='concat'),  # Function that generates updated values
        args={'delim': '-'},  # Parameters (in addition to the input columns) for the update function
        sources=['Locality', 'FIPS']  # Names of input columns (if different from output column)
    )

In [8]:
df.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths
0,03/17/2020,51001,Accomack-51001,Eastern Shore,0,0,0
1,03/17/2020,51003,Albemarle-51003,Thomas Jefferson,0,0,0
2,03/17/2020,51005,Alleghany-51005,Alleghany,0,0,0
3,03/17/2020,51007,Amelia-51007,Piedmont,0,0,0
4,03/17/2020,51009,Amherst-51009,Central Virginia,0,0,0


In [9]:
# Insert a new column from multiple input columns.

df = db\
    .dataset('covid-cases')\
    .insert(
        names='VDH Health District-FIPS',  # Column that is being inserted
        values=db.library.functions().get(name='concat'),  # Function that generates column values
        args={'delim': '/'},  # Parameters (in addition to the input columns) for the column values function
        sources=['VDH Health District', 'FIPS']  # Names of input columns (if different from output column)
    )

In [10]:
df.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,VDH Health District-FIPS
0,03/17/2020,51001,Accomack-51001,Eastern Shore,0,0,0,Eastern Shore/51001
1,03/17/2020,51003,Albemarle-51003,Thomas Jefferson,0,0,0,Thomas Jefferson/51003
2,03/17/2020,51005,Alleghany-51005,Alleghany,0,0,0,Alleghany/51005
3,03/17/2020,51007,Amelia-51007,Piedmont,0,0,0,Piedmont/51007
4,03/17/2020,51009,Amherst-51009,Central Virginia,0,0,0,Central Virginia/51009


In [11]:
# Split VDH Health District on first occurrence of a blank space
# character. Insert the resulting values as two new columns.

df = db\
    .dataset('covid-cases')\
    .insert(
        names=['VDH Health', 'District'],  # Column that is being inserted
        values=db.library.functions().get(name='split'),  # Function that generates column values
        args={'delim': ' '},  # Parameters (in addition to the input columns) for the column values function
        sources=['VDH Health District']  # Names of input columns (if different from output column)
    )

In [12]:
df.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,VDH Health District-FIPS,VDH Health,District
0,03/17/2020,51001,Accomack-51001,Eastern Shore,0,0,0,Eastern Shore/51001,Eastern,Shore
1,03/17/2020,51003,Albemarle-51003,Thomas Jefferson,0,0,0,Thomas Jefferson/51003,Thomas,Jefferson
2,03/17/2020,51005,Alleghany-51005,Alleghany,0,0,0,Alleghany/51005,Alleghany,None
3,03/17/2020,51007,Amelia-51007,Piedmont,0,0,0,Piedmont/51007,Piedmont,None
4,03/17/2020,51009,Amherst-51009,Central Virginia,0,0,0,Central Virginia/51009,Central,Virginia
